In [1]:
from sklearn.utils import shuffle
from scipy import sparse
from time import time

import tensorflow as tf
import pandas as pd
import numpy as np
import random
import heapq
import math
import sys

## Preprocessing

In [2]:
path = 'data/'
dataset = 'ml-1m'
epochs = 100
batch_size = 256
num_factors = 32
reg_mf = 0.0
layers = [128,64,64,32]
reg_layers = [0.,0.,0.,0.]
num_neg = 4
lr = 0.001
patience = 0
max_patience = 5
topK = 10

In [3]:
## load train data 
### user, item, rating, time 으로 구성된 raw data로 부터
### (user, item) value 1인 implicit feedback dictionary로 변환
trn = pd.read_csv(path+dataset+".train.rating", sep="\t", names=['user','item','rating','time'])
num_users=max(trn.user) + 1
num_items=max(trn.item) + 1
trn = trn[trn['rating']>0]

In [4]:
## load test data
### testRatings는 [user, item] list
tst = pd.read_csv("data/ml-1m.test.rating", sep="\t", names=['user','item'], usecols = [0,1])
testRatings = tst.values.tolist()
### testNegatives는 [item_list]이고 index는 user와 동일
neg = pd.read_csv("data/ml-1m.test.negative", sep="_", names=['neg_list'])
testNegatives = neg.neg_list.map(lambda x: list(map(int, x.split('\t')[1:])))

In [5]:
print(trn[:3])
print(testRatings[0:5])
print(testNegatives[0])

   user  item  rating       time
0     0    32       4  978824330
1     0    34       4  978824330
2     0     4       5  978824291
[[0, 25], [1, 133], [2, 207], [3, 208], [4, 222]]
[1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]


In [6]:
## Train Negative sampling and Make Input Data
### user 별 item list 데이터를 만든다.
trn_ = trn.groupby('user').item.agg(lambda x: list(x))
### user 별 item list에서 test set의 item이 아닌 item 중에서 negative item sampling을 수행한다.
item_input = []
labels = []
for u, (l, n, t) in enumerate(zip(trn_, testNegatives, testRatings)):
    ## l은 train item, n은 test neg item, t[1]은 tst item으로
    ## train & test set에 포함되지 않는 item 중 num_neg 배수만큼 추출한다.
    ## 즉, 1번 user가 100개의 item이 있다면 neg item은 400개를 추출 하는 것이다.
    neg_set = list(set(range(num_items)) - set(l + n + [t[1]]))
    neg_set = random.sample(neg_set, min(len(l)*num_neg, len(neg_set)))
    ## train positive, negative item list를 구성하고 그에 맞게 label list를 구성한다.
    item_input.append(l + neg_set)
    labels.append([1]*len(l) + [0]*len(neg_set))
### item_input에 맞에 user_input을 구성한다. 
user_input = [[i]*len(l) for i, l in enumerate(item_input)]

In [7]:
## 2d list to 1d list
user_input = [j for sub in user_input for j in sub]
item_input = [j for sub in item_input for j in sub]
labels = [j for sub in labels for j in sub]

## Prepare Model

In [8]:
## placeholder
user_input_ph = tf.placeholder(shape=[None], dtype=tf.int32, name='user_input')
item_input_ph = tf.placeholder(shape=[None], dtype=tf.int32, name='itme_input')
labels_ph = tf.placeholder(shape=[None], dtype=tf.int32, name='labels')

In [9]:
## user and item embedding matrix
### mlp embedding의  첫번째 layer 노드 수의 1/2씩을 user, item의 embedding size로 생성
with tf.variable_scope("gmf_embedding", reuse=tf.AUTO_REUSE):
    MF_Embedding_User = tf.get_variable(name="mf_embedding_users", shape=[num_users, num_factors],
                                    initializer=tf.contrib.layers.xavier_initializer(),
                                    regularizer=tf.contrib.layers.l2_regularizer(reg_mf))
    MF_Embedding_Item = tf.get_variable(name="mf_embedding_items", shape=[num_items, num_factors],
                                    initializer=tf.contrib.layers.xavier_initializer(),
                                    regularizer=tf.contrib.layers.l2_regularizer(reg_mf))
    
with tf.variable_scope("mlp_embedding", reuse=tf.AUTO_REUSE):
    MLP_Embedding_User = tf.get_variable(name="mlp_embedding_users", shape=[num_users, layers[0]/2],
                                    initializer=tf.contrib.layers.xavier_initializer(),
                                    regularizer=tf.contrib.layers.l2_regularizer(reg_layers[0]))
    MLP_Embedding_Item = tf.get_variable(name="mlp_embedding_items", shape=[num_items, layers[0]/2],
                                    initializer=tf.contrib.layers.xavier_initializer(),
                                    regularizer=tf.contrib.layers.l2_regularizer(reg_layers[0]))   

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.


In [10]:
## prediction for user, item set
### MF part
mf_user_latent = tf.nn.embedding_lookup(MF_Embedding_User, user_input_ph)
mf_item_latent = tf.nn.embedding_lookup(MF_Embedding_Item, item_input_ph)
mf_vector = tf.multiply(mf_user_latent, mf_item_latent)

### MLP part 
mlp_user_latent = tf.nn.embedding_lookup(MLP_Embedding_User, user_input_ph)
mlp_item_latent = tf.nn.embedding_lookup(MLP_Embedding_Item, item_input_ph)
mlp_vector = tf.concat([mlp_user_latent, mlp_item_latent], 1) 
num_layer = len(layers)
for idx in range(1, num_layer):
    mlp_vector = tf.contrib.layers.fully_connected(
        mlp_vector, layers[idx], activation_fn=tf.nn.relu,
        weights_regularizer=tf.contrib.layers.l2_regularizer(reg_layers[idx])
    )
    
### concat MF, MLP
predict_vector = tf.concat([mf_vector, mlp_vector], 1) 

### final prediction layer
prediction = tf.contrib.layers.fully_connected(predict_vector, 1, activation_fn=None)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.


In [11]:
## loss and optimizer
### label shape을 prediction과 동일하게 [None, 1]로 변경해주고, loss(cost)를 계산한다.
loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.cast(tf.reshape(labels_ph, [-1,1]), tf.float32), 
            logits=prediction)
)
## Adam Optimizer 
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

## evaluate

In [12]:
### test set의 구성은 user별 positive item 1개와 negetive item 99개로 구성되어 있다.
### evaluaton process
### 1. 유저별 neg, pos 100개의 item에 대하여 model에 input하여 prediction을 계산한다.
### 2. 100개의 prediction 값 중 내림차순으로 상위 topK(10개) item을 순서를 유지하여 추출한다.
### 3. HitRatio는 10개 중 positive item이 있으면 1 없으면 0으로 값을 return한다.
### 4. NDCG는 positive item의 순위를 고려하여 계산한것이다. 즉, rank 1이라면 값은 1이 되고,
###    rank가 2, 3... 뒤로 갈 수록 log비율로 감소시킨다. 10개 중 없으면 0을 리턴함.
### 5. user별 HitRatio와 NDCG 값을 list로 구성하고 평균을 구하면 최종 평가 지표가 된다. 
def evaluation(sess, testRatings, testNegatives, K):
    def getHitRatio(ranklist, gtItem):
        for item in ranklist:
            if item == gtItem:
                return 1
        return 0

    def getNDCG(ranklist, gtItem):
        for i in range(len(ranklist)):
            item = ranklist[i]
            if item == gtItem:
                return math.log(2) / math.log(i+2)
        return 0
    
    hits, ndcgs = [],[]
    for i in range(len(testRatings)):
        rating = testRatings[i]
        items = testNegatives[i]
        u = rating[0]
        gtItem = rating[1]
        items.append(gtItem)
        
        # Get prediction scores
        map_item_score = {}
        users = np.full(len(items), u, dtype = 'int32')
        predictions = sess.run(prediction, 
                               feed_dict={user_input_ph: users, item_input_ph: items})

        for i in range(len(items)):
            item = items[i]
            map_item_score[item] = predictions[i]
        items.pop()
        
        # Evaluate top rank list
        ranklist = heapq.nlargest(K, map_item_score, key=map_item_score.get)
        hr = getHitRatio(ranklist, gtItem)
        ndcg = getNDCG(ranklist, gtItem)
        
        hits.append(hr)
        ndcgs.append(ndcg)
    
    return(hits, ndcgs)

## Training

In [13]:
## session start
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
## initial HitRatio and NDCG evaluation
hits, ndcgs = evaluation(sess, testRatings, testNegatives, topK)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print('Init: HR = %.4f, NDCG = %.4f' % (hr, ndcg))

Init: HR = 0.0945, NDCG = 0.0425


In [15]:
## Train model
### early stopping을 위한 값 세팅
best_hr, best_ndcg, best_iter = hr, ndcg, -1
for epoch in range(epochs):
    t1 = time()
    ### data shuffle
    user_input, item_input, labels = shuffle(
        user_input, item_input, labels
    )
    ### training
    batch_len = len(user_input) // batch_size
    batch_loss = list()
    for i in range(batch_len):
        start = i*batch_size
        user_input_bc = user_input[start : start+batch_size]
        item_input_bc = item_input[start : start+batch_size]
        labels_bc = labels[start : start+batch_size]
    
        _, l = sess.run([train_op, loss], feed_dict={
            user_input_ph: user_input_bc,
            item_input_ph: item_input_bc,
            labels_ph: labels_bc
        })
        batch_loss.append(l)
        
        if i % 1000==0:
            print("epochs:", epoch, "batchs:", i, "\n", "mean_loss:", 
                  np.array(batch_loss).mean())
    
    ### evaluation
    hits, ndcgs = evaluation(sess, testRatings, testNegatives, topK)
    hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    print('eopch', epoch, '- HR = %.4f, NDCG = %.4f' % (hr, ndcg), 
          'elapsed time: %.2f' % (time()-t1))
    ### HitRatio가 이전 값보다 작아지면 종료한다.
    if hr > best_hr:
        best_hr, best_ndcg, best_iter = hr, ndcg, epoch
        patience = 0
    else:
        patience += 1
        if patience == max_patience:
            print("max patience, training stop")
            print("max hit ratio:", best_hr, "max ndcg ratio:", best_ndcg)
            break

epochs: 0 batchs: 0 
 mean_loss: 0.6923248
epochs: 0 batchs: 1000 
 mean_loss: 0.39575312
epochs: 0 batchs: 2000 
 mean_loss: 0.3831697
epochs: 0 batchs: 3000 
 mean_loss: 0.37642172
epochs: 0 batchs: 4000 
 mean_loss: 0.36915818
epochs: 0 batchs: 5000 
 mean_loss: 0.36150393
epochs: 0 batchs: 6000 
 mean_loss: 0.3548257
epochs: 0 batchs: 7000 
 mean_loss: 0.34877303
epochs: 0 batchs: 8000 
 mean_loss: 0.3437354
epochs: 0 batchs: 9000 
 mean_loss: 0.33887035
epochs: 0 batchs: 10000 
 mean_loss: 0.3348629
epochs: 0 batchs: 11000 
 mean_loss: 0.3311982
epochs: 0 batchs: 12000 
 mean_loss: 0.32791173
epochs: 0 batchs: 13000 
 mean_loss: 0.32481402
epochs: 0 batchs: 14000 
 mean_loss: 0.32216352
epochs: 0 batchs: 15000 
 mean_loss: 0.31972203
epochs: 0 batchs: 16000 
 mean_loss: 0.31740117
epochs: 0 batchs: 17000 
 mean_loss: 0.31521845
epochs: 0 batchs: 18000 
 mean_loss: 0.31320068
eopch 0 - HR = 0.6273, NDCG = 0.3611 elapsed time: 73.97
epochs: 1 batchs: 0 
 mean_loss: 0.240859
epochs: 

epochs: 8 batchs: 13000 
 mean_loss: 0.20143138
epochs: 8 batchs: 14000 
 mean_loss: 0.2021008
epochs: 8 batchs: 15000 
 mean_loss: 0.20275941
epochs: 8 batchs: 16000 
 mean_loss: 0.20336038
epochs: 8 batchs: 17000 
 mean_loss: 0.20397489
epochs: 8 batchs: 18000 
 mean_loss: 0.20463163
eopch 8 - HR = 0.6487, NDCG = 0.3799 elapsed time: 74.13
epochs: 9 batchs: 0 
 mean_loss: 0.15843788
epochs: 9 batchs: 1000 
 mean_loss: 0.18282816
epochs: 9 batchs: 2000 
 mean_loss: 0.18497096
epochs: 9 batchs: 3000 
 mean_loss: 0.18593818
epochs: 9 batchs: 4000 
 mean_loss: 0.18773863
epochs: 9 batchs: 5000 
 mean_loss: 0.18935783
epochs: 9 batchs: 6000 
 mean_loss: 0.19073859
epochs: 9 batchs: 7000 
 mean_loss: 0.19180205
epochs: 9 batchs: 8000 
 mean_loss: 0.19266301
epochs: 9 batchs: 9000 
 mean_loss: 0.19374803
epochs: 9 batchs: 10000 
 mean_loss: 0.19471754
epochs: 9 batchs: 11000 
 mean_loss: 0.19567364
epochs: 9 batchs: 12000 
 mean_loss: 0.19632834
epochs: 9 batchs: 13000 
 mean_loss: 0.197203